In [1]:
# Crea un netcdf con el viento a 925 hPa alineado con respecto
# a la fecha del onset de la temporada de lluvias.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

# Datos básicos.

path_d = "../data/ERA5/"
path_r = "../results/onset/"

# Cantidad de días a promediar.
d = 20

# Variable, nivel de presión y región.
vars = [ "u", "v", "gp", "sst", "olr", "sp",
    "vidmf", "vivfu", "vivfv", "VIDMFI" ]
levels = [ "925", "200" ]
region = [ "mexico" ]

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Cargamos la información de desfase para cada año.
shift = []
delta = []
shift.append ( pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True) )
shift.append( pd.read_csv(path_r + "shift_withdrawal_CHIRPS.csv",
    index_col = "Año", squeeze = True) )
# Fecha pivote del desfase. 
for i in [0, 1]:
    delta.append( shift[i].max() )
    shift[i] = delta[i] - shift[i]

onset = [ "onset", "withdrawal" ]
anom = [ "", "_anom" ]

In [ ]:
# Temperatura superficial del océano.
v = 3

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Temperatura superficial del océano.
v = 3

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# OLR
v = 4

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# OLR
v = 4

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Presión superficial
v = 5

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Presión superficial
v = 5

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Vertical integral of moisture flow
v = 6

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Vertical integral of moisture flow
v = 6

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Tranporte zonal de vapor
v = 7

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Tranporte zonal de vapor
v = 7

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Tranporte meridional de vapor
v = 8

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Tranporte meridional de vapor
v = 8

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Vertically integrated moisture flow
v = 9

# Normal.
a = anom[0]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Vertically integrated moisture flow
v = 9

# Anomalías
a = anom[1]
fname = region[0] + "_" + vars[v] + "_daily" + a + ".grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )
# Media móvil de precipitación centrada a d días.
ds = ( ds.rolling(time = d, center = True).mean() )
# Guardamos el netcdf.
ds.to_netcdf( path_r + "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_" + vars[v]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )